In [1]:
%%bash
# Need to install pyspark
# if pyspark is already installed, will print a message indicating pyspark already isntalled
pip install pyspark


  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=894faeb6c0bb654ecc87c7ef89e413594b4f23994d684efdfceb6a6f9691b845
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
%matplotlib inline
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
import os
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
sampled_df = spark.read.format("csv").option("header", "true").load("drive/Shared drives/IST 718 Group1/sampled_used_cars_data_nodesc.csv")


In [4]:
sampled_df.describe().show()

+-------+------------+-----+----------+----------+-----------+-----------+-----------+-----------------+--------------------+------------------+------------------+--------------------+-------------------+--------------------+------------------+------+-------------+----------------+--------------+-------------+----------------+---------+-------------+-------+--------------------+------------------+--------------------+------+------------+------+------+---------+-----------------+------+-----------+-------------+--------------------+------------------+--------------------+--------------------+----------+---------------+------------------+------------------+------------------+------------------+------------------+-------+------------------+------------------+------------------+--------------------+-----------+--------------------+------------+--------------------+------+------------------+-----------------------+------------+--------------------+---------+-------+------------------+
|summ

In [5]:
sampled_df.limit(10).toPandas()

,back_legroom,bed,bed_height,bed_length,body_type,cabin,city,city_fuel_economy,combine_fuel_economy,daysonmarket,dealer_zip,engine_cylinders,engine_displacement,engine_type,exterior_color,fleet,frame_damaged,franchise_dealer,franchise_make,front_legroom,fuel_tank_volume,fuel_type,has_accidents,height,highway_fuel_economy,horsepower,interior_color,isCab,is_certified,is_cpo,is_new,is_oemcpo,latitude,length,listed_date,listing_color,listing_id,longitude,main_picture_url,major_options,make_name,maximum_seating,mileage,model_name,owner_count,power,price,salvage,savings_amount,seller_rating,sp_id,sp_name,theft_title,torque,transmission,transmission_display,trimId,trim_name,vehicle_damage_category,wheel_system,wheel_system_display,wheelbase,width,year
0,35.1 in,None,None,None,SUV / Crossover,None,Bayamon,None,None,522,960,I4,1300,I4,Solar Yellow,None,None,TRUE,Jeep,41.2 in,12.7 gal,Gasoline,None,66.5 in,None,177,Black,None,None,None,TRUE,None,18.3988,166.6 in,4/6/2019,YELLOW,237132766,-66.1582,https://static.cargurus.com/images/forsale/202...,['Quick Order Package'],Jeep,5 seats,7,Renegade,None,"177 hp @ 5,750 RPM",23141,None,0,2.8,370599,Flagship Chrysler,None,"200 lb-ft @ 1,750 RPM",A,9-Speed Automatic Overdrive,t83804,Latitude FWD,None,FWD,Front-Wheel Drive,101.2 in,79.6 in,2019
1,38.1 in,None,None,None,SUV / Crossover,None,San Juan,None,None,207,922,I4,2000,I4,Narvik Black,None,None,TRUE,Land Rover,39.1 in,17.7 gal,Gasoline,None,68 in,None,246,Black (Ebony),None,None,None,TRUE,None,18.4439,181 in,2/15/2020,BLACK,265946296,-66.0785,https://static.cargurus.com/images/forsale/202...,['Adaptive Cruise Control'],Land Rover,7 seats,8,Discovery Sport,None,"246 hp @ 5,500 RPM",46500,None,0,3,389227,Land Rover San Juan,None,"269 lb-ft @ 1,400 RPM",A,9-Speed Automatic Overdrive,t86759,S AWD,None,AWD,All-Wheel Drive,107.9 in,85.6 in,2020
2,33.4 in,None,None,None,SUV / Crossover,None,Tampa,25,None,194,33614,I4,2000,I4,Brilliant Silver Metallic,None,None,TRUE,None,42.8 in,14.5 gal,Gasoline,None,62.5 in,32,141,Black (Charcoal),None,None,None,TRUE,None,27.9959,172.8 in,2/29/2020,SILVER,267331641,-82.5076,https://static.cargurus.com/images/forsale/202...,"['Steel Wheels', 'Bluetooth', 'Backup Camera',...",Nissan,5 seats,2,Rogue Sport,None,"141 hp @ 6,000 RPM",21466,None,0,4.233333333,377115,Courtesy Nissan of Tampa,None,"147 lb-ft @ 4,400 RPM",CVT,Continuously Variable Transmission,t89963,S FWD,None,FWD,Front-Wheel Drive,104.2 in,72.3 in,2020
3,--,None,None,None,Coupe,None,Phoenix,None,None,80,85034,H6,3800,H6,Guards Red,FALSE,FALSE,FALSE,None,--,16.9 gal,Gasoline,FALSE,51.2 in,None,355,Black,FALSE,None,None,FALSE,None,33.4225,174.3 in,6/23/2020,RED,274772549,-111.989,https://static.cargurus.com/images/forsale/202...,"['Leather Seats', 'Backup Camera']",Porsche,4 seats,82632,911,5,"355 hp @ 6,600 RPM",40950,FALSE,5077,4.642857143,379334,Distinctive Auto Brokers,FALSE,"295 lb-ft @ 4,600 RPM",A,6-Speed Automatic,t33302,Carrera S,None,RWD,Rear-Wheel Drive,92.5 in,71.2 in,2008
4,33.7 in,None,None,None,Coupe,None,Knoxville,22,None,36,37912,I6,3000,I6,Estoril Blue Metallic,FALSE,FALSE,TRUE,Chevrolet,41.8 in,15.8 gal,Gasoline,FALSE,54.2 in,32,300,Black,FALSE,None,None,FALSE,None,35.9991,182.6 in,8/5/2020,BLUE,278419497,-83.9644,https://static.cargurus.com/images/forsale/202...,"['Sport Package', 'Driver Assistance Package',...",BMW,4 seats,78975,4 Series,2,"300 hp @ 5,800 RPM",20993,FALSE,1024,4.555555556,53420,Reeder Chevrolet,FALSE,"300 lb-ft @ 1,300 RPM",A,Automatic,t52660,435i Coupe RWD,None,RWD,Rear-Wheel Drive,110.6 in,79.4 in,2014
5,None,None,None,None,Pickup Truck,None,Albuquerque,None,None,16,87112,V8 Biodiesel,None,V8 Biodiesel,Oxford White,None,None,TRUE,Ford,None,None,Biodiesel,None,None,None,None,None,None,None,None,TRUE,None,35.0872,None,8/26/2020,WHITE,280318985,-106.548,https://static.cargurus.com/images/forsale/202...,"['Value Package', 'Steel Wheels', 'Bluetooth',...",Ford,None,0,F-450 Super Duty,None,None,53355,None,0,4,56927,Rich Fo

In [7]:
# EDA: Dimension Inspection
from pyspark.sql import functions as fn # import functions
print("Number of Observations:",sampled_df.count())
print("Number of Columns:",len(sampled_df.columns))
print("Number of Columns with real data (less the index column):",(len(sampled_df.columns)-1))

Number of Observations: 600000
Number of Columns: 64
Number of Columns with real data (less the index column): 63


In [13]:
# EDA: Data Type Inspection
sampled_df.printSchema()

root
 |-- back_legroom: string (nullable = true)
 |-- bed: string (nullable = true)
 |-- bed_height: string (nullable = true)
 |-- bed_length: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- cabin: string (nullable = true)
 |-- city: string (nullable = true)
 |-- city_fuel_economy: string (nullable = true)
 |-- combine_fuel_economy: string (nullable = true)
 |-- daysonmarket: string (nullable = true)
 |-- dealer_zip: string (nullable = true)
 |-- engine_cylinders: string (nullable = true)
 |-- engine_displacement: string (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- exterior_color: string (nullable = true)
 |-- fleet: string (nullable = true)
 |-- frame_damaged: string (nullable = true)
 |-- franchise_dealer: string (nullable = true)
 |-- franchise_make: string (nullable = true)
 |-- front_legroom: string (nullable = true)
 |-- fuel_tank_volume: string (nullable = true)
 |-- fuel_type: string (nullable = true)
 |-- has_accidents: string (nulla

In [11]:
# EDA: Duplicate Data Removal
sampled_df = sampled_df.distinct()

In [ ]:
# Generate of list of column names for future use
colList = sampled_df.columns
numColList = []
catColList = []